In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [68]:
import os
import time
import json
import pandas as pd
from tqdm import tqdm

In [69]:
root = "drive/MyDrive/data/bank/"
source_dir = "/extended/"
output_dir = "/extended/"
url_bamosz = "https://www.bamosz.hu/alapoldal?isin="

In [70]:
with open(root + "data.json","r",encoding="utf8") as jf:
    stocks = json.load(jf)

In [71]:
for key in tqdm(stocks.keys()):
  file_with_paht = root + source_dir + key + ".csv"
  if os.path.exists(file_with_paht):
    stocks[key] = {
        "data" : pd.read_csv(file_with_paht, sep=";", decimal=",",encoding='utf-8-sig'),
        "name" : stocks[key]
    }
  else:
    stocks[key] = { "name" : stocks[key] }

100%|██████████| 93/93 [00:01<00:00, 71.66it/s]


In [72]:
extended = {}
while len(list(stocks.keys())) > 0: 
  errors = {}
  pbar = tqdm(stocks.keys())
  for key in pbar:    
    if 'data' in stocks[key].keys():
      try:      
        stocks[key]["new_data"] = pd.read_html(url_bamosz+key, 
                                              attrs={"class" : "dataTable2"}, 
                                              decimal=",", 
                                              thousands='.')[1]
        extended[key] = stocks[key].copy()
        pbar.set_description("Completed: " + key)
      except:
        errors[key] = stocks[key].copy()
        pbar.set_description("Errors: " + key)
      time.sleep(1)

  stocks = errors.copy()
  time.sleep(2)

Completed: HU0000709092: 100%|██████████| 93/93 [04:15<00:00,  2.75s/it]


In [73]:
def cleaner(item):
  return item.replace("%","").replace(",",".").replace(" ","")

def rename_columns(data):
  return data.rename(columns={"Nettó eszközérték": "NEÉ",
                              "Napi bef. jegy forgalom": "Napi befjegy. forgalom",
                              "Napi befjegy forgalom vált." : "Napi befjegy. forgalom változás(%)",
                              "3 havi" : "3 hónapos hozam",
                              "6 havi" : "6 hónapos hozam",
                              "1 éves" : "1 éves hozam",
                              "3 éves" : "3 éves hozam",
                              "5 éves" : "5 éves hozam",
                              "10 éves" : "10 éves hozam"})
  
def reformat_csv(data):
  tmp = data.copy()
  tmp.columns = tmp.values[0]
  tmp = tmp[1:].copy().iloc[::-1]
  tmp = tmp.fillna(0.0)
  tmp["Dátum"] = tmp["Dátum"].apply(lambda x: x[:4] + "-" + x[4:6] + "-" + x[6:])
  tmp["Árfolyam"] = tmp["Árfolyam"].apply(lambda x: float(cleaner(cleaner(str(x)))))
  tmp["Nettó eszközérték"] = tmp["Nettó eszközérték"].apply(lambda x: int(cleaner(str(x))))
  tmp["Napi bef. jegy forgalom"] = tmp["Napi bef. jegy forgalom"].apply(lambda x: float(x))
  tmp["Napi befjegy forgalom vált."] = tmp["Napi befjegy forgalom vált."].apply(lambda x: float(cleaner(str(x))))
  tmp["3 havi"] = tmp["3 havi"].apply(lambda x: float(cleaner(str(x))))
  tmp["6 havi"] = tmp["6 havi"].apply(lambda x: float(cleaner(str(x))))
  tmp["1 éves"] = tmp["1 éves"].apply(lambda x: float(cleaner(str(x))))
  tmp["3 éves"] = tmp["3 éves"].apply(lambda x: float(cleaner(str(x))))
  tmp["5 éves"] = tmp["5 éves"].apply(lambda x: float(cleaner(str(x))))
  tmp["10 éves"] = tmp["10 éves"].apply(lambda x: float(cleaner(str(x))))
  tmp = rename_columns(tmp)
  return tmp

In [74]:
# import copy
# ext2 = copy.deepcopy(extended)

for key in tqdm(extended.keys()):
  extended[key]["new_data"] = reformat_csv(extended[key]["new_data"])

  dates = list(extended[key]["data"]["Dátum"].values)
  tmp = extended[key]["new_data"][~extended[key]["new_data"]["Dátum"].isin(dates)]
  extended[key]["ext_data"] = pd.concat([tmp.iloc[::-1], extended[key]["data"]])
  extended[key]["ext_data"] = extended[key]["ext_data"][~(extended[key]["ext_data"]["Árfolyam"] == 0)]
  extended[key]["ext_data"]["id"] = [key] * len(extended[key]["ext_data"])
  extended[key]["ext_data"]["name"] = [extended[key]["name"]] * len(extended[key]["ext_data"])

100%|██████████| 91/91 [00:01<00:00, 68.03it/s]


In [75]:
def get_bollinger_bands(prices, rate=20):
    sma = prices.rolling(rate).mean()
    std = prices.rolling(rate).std()
    bollinger_up = sma + std * 2 # Calculate top band
    bollinger_down = sma - std * 2 # Calculate bottom band
    
    return bollinger_up, bollinger_down

for key in tqdm(extended.keys()):
  extended[key]["ext_data"] = extended[key]["ext_data"].sort_values(by='Dátum')
  extended[key]["ext_data"]["Árfolyam_MA7"] = extended[key]["ext_data"]["Árfolyam"].rolling(7).mean()
  extended[key]["ext_data"]["Árfolyam_MA14"] = extended[key]["ext_data"]["Árfolyam"].rolling(14).mean()
  extended[key]["ext_data"]["Árfolyam_MA21"] = extended[key]["ext_data"]["Árfolyam"].rolling(21).mean()
  extended[key]["ext_data"]["Árfolyam_MA30"] = extended[key]["ext_data"]["Árfolyam"].rolling(30).mean()
  
  bollinger_up, bollinger_down = get_bollinger_bands(extended[key]["ext_data"]["Árfolyam"])
  extended[key]["ext_data"]["bollinger_up_20"] = bollinger_up
  extended[key]["ext_data"]["bollinger_down_20"] = bollinger_down

  extended[key]["ext_data"] = extended[key]["ext_data"].fillna(0.0)

100%|██████████| 91/91 [00:00<00:00, 96.25it/s] 


In [76]:
for key in tqdm(extended.keys()):
  extended[key]["ext_data"].to_csv(root + output_dir + key + ".csv",
                                   sep=";",
                                   index=False,
                                   encoding='utf-8-sig',
                                   decimal=",")

100%|██████████| 91/91 [00:10<00:00,  8.74it/s]


In [77]:
total = pd.concat([extended[key]["ext_data"] for key in tqdm(extended.keys())])
total.to_csv(root + "total.csv",
             sep=";",
             index=False,
             encoding='utf-8-sig',
             decimal=",")
total.info()

100%|██████████| 91/91 [00:00<00:00, 243886.05it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 204391 entries, 1227 to 1
Data columns (total 20 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Dátum                               204391 non-null  object 
 1   Árfolyam                            204391 non-null  float64
 2   NEÉ                                 204391 non-null  float64
 3   Kifizetett hozam                    204391 non-null  object 
 4   Napi befjegy. forgalom              204391 non-null  float64
 5   Napi befjegy. forgalom változás(%)  204391 non-null  float64
 6   3 hónapos hozam                     204391 non-null  float64
 7   6 hónapos hozam                     204391 non-null  float64
 8   1 éves hozam                        204391 non-null  float64
 9   3 éves hozam                        204391 non-null  float64
 10  5 éves hozam                        204391 non-null  float64
 11  10 éves hozam               